In [37]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [38]:
# # --- MCP Resources ---
# @mcp.resource("frameworks://supported")
# def supported_frameworks() -> List[str]:
#     """Get list of supported frameworks"""
#     return ["react", "vue", "angular", "svelte"]
# 
# @mcp.resource("examples://descriptions/{framework}")
# def framework_examples(framework: str) -> List[str]:
#     """Get example app descriptions for a specific framework"""
#     examples = {
#         "react": [
#             "Todo app with local storage",
#             "Weather dashboard with API integration",
#             "E-commerce product page with cart functionality"
#         ],
#         "vue": [
#             "Blog with markdown support",
#             "Task management app with drag-and-drop",
#             "Real-time chat interface"
#         ]
#     }
#     return examples.get(framework.lower(), ["No examples available"])

In [39]:
import asyncio
from langchain_mcp_tools import convert_mcp_to_langchain_tools
from langchain.agents import AgentExecutor
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

async def main():
    # Define the MCP servers
    mcp_servers = {
        "app_generator": {
            "command": "python",
            "args": ["bolt_app_generator.py"]
        }
    }
    
    # Convert to LangChain tools
    tools, cleanup = await convert_mcp_to_langchain_tools(mcp_servers)
    
    for tool in tools:
        print("Available tool:")
        print(f"- {tool.name}")

    try:
        llm = init_chat_model(
            model="gpt-4.1-nano",
            temperature=0.5,
            openai_api_key=OPENAI_API_KEY
        )
        
        # Create ReAct agent with proper system message
        agent = create_react_agent(
            model=llm, 
            tools=tools,
            prompt="You are a helpful assistant that creates application scaffolds. Use the available tools to generate code based on user requests."
        )
        
        # Run the agent
        result = await agent.ainvoke({
            "messages": "Create a React dashboard for monitoring daily tasks. It should show tasks by priority and deadline."
        })
        
        print("Agent response:")
        
    finally:
        # Clean up MCP servers
        await cleanup()

In [40]:
result = await main()

[INFO] MCP server "app_generator": initializing with: {'command': 'python', 'args': ['bolt_app_generator.py']}
[INFO] MCP server "app_generator": connected
[INFO] MCP server "app_generator": 1 tool(s) available:
[INFO] - generate_app_as_xml
[INFO] MCP servers initialized: 1 tool(s) available in total


Available tool:
- generate_app_as_xml


[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] MCP tool "app_generator"/"generate_app_as_xml" received input: {'description': 'A React dashboard for monitoring daily tasks, showing tasks categorized by priority and deadline.', 'framework': 'react'}
[INFO] MCP tool "app_generator"/"generate_app_as_xml" received result (size: 4578)
[INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[INFO] MCP server "app_generator": session closed


Agent response:


In [42]:
result